In [63]:
import pandas as pd
df=pd.read_csv("article_prices.csv.gz",compression="gzip")


In [64]:
df.head()

,product,offer,supplier,tier,price
0,1,1,1498,1.0,24.12
1,1,2,1690,1.0,24.14
2,1,2,1690,5.0,24.10
3,1,3,1955B,1.0,26.00
4,1,3,1955B,2.0,25.87


In [65]:
df.shape

(21649, 5)

In [66]:
df.nunique()

product      777
offer       5771
supplier       9
tier         129
price       9067
dtype: int64

In [67]:
df["supplier"].unique() #there are 9 suppliers not 8 as mentioned in task description.

array(['1498', '1690', '1955B', '2478B', '4757', '5417B', '5766', '9587',
       '2627A'], dtype=object)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21649 entries, 0 to 21648
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   product   21649 non-null  int64  
 1   offer     21649 non-null  int64  
 2   supplier  21649 non-null  object 
 3   tier      21649 non-null  float64
 4   price     21649 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 845.8+ KB


In [69]:
df.describe()

,product,offer,tier,price
count,21649.000000,21649.000000,21649.000000,21649.000000
mean,378.133124,2842.434662,55.016983,50.088016
std,219.809305,1634.171546,658.844792,93.210518
min,1.000000,1.000000,0.333340,0.025140
25%,189.000000,1440.000000,2.000000,7.570000
50%,375.000000,2845.000000,5.000000,19.100000
75%,566.000000,4242.000000,10.000000,47.760000
max,777.000000,5771.000000,50000.000000,1114.600000


In [70]:
df[df["tier"]%1!=0]

,product,offer,supplier,tier,price
1271,43,330,5417B,0.33334,8.76
1272,43,330,5417B,0.66667,7.83
1273,43,330,5417B,3.33334,6.69
1274,43,330,5417B,33.33334,6.45


##### so there are also fraction quantity.

In [71]:
def tier_range(df_group):
    tier_unique=sorted(df_group["tier"].unique())
    prev=None
    tier_range={}
    for tier in tier_unique:
        if prev is not None:
            tier_range[prev]=[prev,tier]
        prev=tier
    tier_range[tier_unique[-1]]=[tier_unique[-1],float("inf")]
    df_group["tier_range"]=df_group["tier"].map(tier_range)
    return df_group
            

In [72]:
df_group=df.groupby(["supplier","product"]).apply(tier_range)

C:\Users\KASHIF AI\AppData\Local\Temp\ipykernel_7256\2571673669.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_group=df.groupby(["supplier","product"]).apply(tier_range)


In [73]:
df_group.reset_index(drop=True,inplace=True)

### Let's try to find best supplier for product 1 for a given Quantity Q

In [128]:
df_product1=df_group[df_group["product"]==1]
df_product1.reset_index(drop=True,inplace=True)
df_product1

,product,offer,supplier,tier,price,tier_range
0,1,1,1498,1.0,24.12,"[1.0, inf]"
1,1,2,1690,1.0,24.14,"[1.0, 5.0]"
2,1,2,1690,5.0,24.10,"[5.0, inf]"
3,1,3,1955B,1.0,26.00,"[1.0, 2.0]"
4,1,3,1955B,2.0,25.87,"[2.0, 5.0]"
5,1,3,1955B,5.0,25.74,"[5.0, inf]"
6,1,4,2478B,2.0,26.73,"[2.0, 3.0]"
7,1,4,2478B,3.0,24.22,"[3.0, 9.0]"
8,1,4,2478B,9.0,24.21,"[9.0, 30.0]"
9,1,4,2478B,30.0,24.20,"[30.0, inf]"


In [75]:
def find_correct_tier(group, Q):
    for idx, row in group.iterrows():
        # Extract tier range from 'tier_range' column
        min_quantity, max_quantity = row['tier_range'][0],row['tier_range'][1]
        # Check if given quantity Q falls within the tier range
        if Q >= min_quantity and (Q < max_quantity or max_quantity == float('inf')):
            return row  # Return the row if correct tier is found
    return None  # Return None if correct tier is not found

In [32]:
Q=10 #let's say we need best supplier that offers best price for a product1 and quantity Q
df_product1_grouped=df_product1.groupby(["product","supplier"]).apply(lambda group:find_correct_tier(group,Q))

C:\Users\KASHIF AI\AppData\Local\Temp\ipykernel_7256\3320901497.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_product1_grouped=df_product1.groupby(["product","supplier"]).apply(lambda group:find_correct_tier(group,Q))


In [34]:
df_product1_grouped.reset_index(drop=True,inplace=True)

In [43]:
df_product1_grouped.sort_values(by="price",ascending=True)

,product,offer,supplier,tier,price,tier_range
5,1,6,5417B,10.0,21.15,"[10.0, 24.0]"
7,1,8,9587,10.0,24.05,"[10.0, inf]"
1,1,2,1690,5.0,24.10,"[5.0, inf]"
0,1,1,1498,1.0,24.12,"[1.0, inf]"
3,1,4,2478B,9.0,24.21,"[9.0, 29.0]"
2,1,3,1955B,5.0,25.74,"[5.0, inf]"
6,1,7,5766,4.0,26.97,"[4.0, inf]"
4,1,5,4757,10.0,28.12,"[10.0, 19.0]"


##### If we look above,supplier '5417B' offers best price per quantity of 21.5 euros for a quantity of 10 of product1

### Now lets make a function that,given product and quantity we need,will return name of supplier who offers best price(per piece) for that product at that quantity.

In [122]:
def find_best_supplier(product,quantity):
    df_prod=df_group[df_group["product"]==product]
    df_prod.reset_index(drop=True,inplace=True)
    df_grp=df_prod.groupby(["product","supplier"])[['product', 'offer', 'supplier', 'tier', 'price','tier_range']]
    df_prod_grouped=df_grp.apply(lambda group:find_correct_tier(group,quantity))
    if len(df_prod_grouped)<1:
        return None
    df_prod_grouped.reset_index(drop=True,inplace=True)
    best_supplier=df_prod_grouped.sort_values(by="price",ascending=True).reset_index(drop=True).loc[0,"supplier"]
    return best_supplier

In [127]:
product=300
Q=40
best_supplier=find_best_supplier(product=product,quantity=Q)
if best_supplier is None:
    print("Tier not available,choose higher qunatity")
else:
    print(f"Best supplier for product {product} at quantity {Q} is: {best_supplier}")

Best supplier for product 300 at quantity 40 is: 1690
